#Importing drive so as to access train and test files from Google Drive

*  Google Colab provides a GPU which helps in training the model faster




In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# **Importing required libraries**
****

In [2]:
import pandas as pd
import numpy as np


import keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.regularizers import L1L2

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


import string
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer, SnowballStemmer


Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


##**Reading Train and Test Files from google drive and transforming them into a pandas dataframe**

In [0]:
train_data = pd.read_csv('gdrive/My Drive/train_file.csv')#, encoding='ISO-8859-1')
test_data = pd.read_csv('gdrive/My Drive/test_file.csv')#, encoding='utf8')

**Changing the names of the columns into a lower case letters and white spaces are being replaced with underscore(''_")**

In [0]:
train_data.columns = train_data.columns.str.lower().str.rstrip()
train_data.columns = train_data.columns.str.replace(' ', '_')

##Data Analysis


*   Checking the length of each string present in the columns, Title and HeadLine, of the train dataframe
*   Checking the existence of Null values in the columns of the train dataset
*   Checking the Count of the NULL Values in each column of the train dataset
*   Shape of the train dataset and test dataset


 

In [5]:
max(train_data["headline"].str.len()), max(train_data["title"].str.len())

(446, 170)

In [6]:
train_data.head(20)

,idlink,title,headline,source,topic,publishdate,facebook,googleplus,linkedin,sentimenttitle,sentimentheadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1,-1,-1,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1,-1,-1,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1,-1,-1,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1,-1,-1,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1,-1,-1,0.000000,0.141084
5,1P7kLrnWEp,Intellitec Solutions to Host 13th Annual Sprin...,Over 100 attendees expected to see latest vers...,PRWeb,microsoft,2015-03-01 00:19:00,-1,-1,-1,-0.075378,0.036773
6,lKg2pImhCl,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",NaN,palestine,2016-02-28 14:03:00,0,0,0,0.000000,-0.005906
7,X2KssRh8hS,"Obama, stars pay a musical tribute to Ray Charles",First lady Michelle Obama speaks in the State ...,Coast Reporter,obama,2015-03-01 00:45:00,-1,-1,-1,0.083333,0.103003
8,xCHOHAl8v8,Fire claims more than 100-year-old barn in Han...,A Hancock County man lost a barn early Monday ...,WTHR Indianapolis,palestine,2015-03-01 01:20:00,-1,-1,-1,-0.173925,-0.050185
9,OR6Xuthveg,Microsoft's new Windows 10 ad targets Apple,"New Delhi, Feb.29 : Technology giant Microsoft...",New Kerala,microsoft,2015-03-01 01:32:00,-1,-1,-1,-0.059536,-0.081715


In [11]:
train_data.isnull().any(), train_data.isnull().sum()

(idlink               False
 title                False
 headline             False
 source               False
 topic                False
 publishdate          False
 facebook             False
 googleplus           False
 linkedin             False
 sentimenttitle       False
 sentimentheadline    False
 dtype: bool, idlink               0
 title                0
 headline             0
 source               0
 topic                0
 publishdate          0
 facebook             0
 googleplus           0
 linkedin             0
 sentimenttitle       0
 sentimentheadline    0
 dtype: int64)

In [0]:
train_data['source'].fillna(value=train_data['source'].value_counts().index[0],inplace =True)

In [0]:
train_data.shape, test_data.shape

## Read the required columns for Predictions




*   Consider only 'Title' column for predictions of SentimentTitle

*   Consider only 'HeadLine' column for predictions of SentimentHeadLine



In [0]:
train_data_title = train_data[train_data['title'].notnull()]
train_data_hLine = train_data[train_data['headline'].notnull()]

In [0]:
X_train_title = train_data_title['title']
Y_train_title = train_data_title['sentimenttitle']
X_train_hLine = train_data_hLine['headline']
Y_train_hLine = train_data_hLine['sentimentheadline']

# Preprocess text



*   Remove punctuations from data.
*   Apply lemmatization on words. Convert each word to its lemma.




---





In [0]:
#stemmer = SnowballStemmer("english")
lemma = WordNetLemmatizer()

def preprocess_data(X_train, lemma):
    preprocessed_data = []
    trans = str.maketrans('—/(){}-', ' ' * 7)
    trans_punc = str.maketrans('', '', string.punctuation)

    for text in X_train:
        #print(text)
        text = text.lower().translate(trans)
        text = text.translate(trans_punc)
        text = [lemma.lemmatize(word) for word in text.split()]
        text = [x for x in text if not any(c.isdigit() for c in x)]
        preprocessed_data.append(' '.join(text))
        #print(text)

    return preprocessed_data

In [0]:
preprocessed_data_t = preprocess_data(X_train_title, lemma)
preprocessed_data_h = preprocess_data(X_train_hLine, lemma)

# Tfidf Vectorizer - Transform text into vector











In [0]:
vectorizer_t = TfidfVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english', ngram_range=(1, 3))
vectorizer_t.fit(X_train_title)

vectorizer_h = TfidfVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english', ngram_range=(1, 3))
vectorizer_h.fit(X_train_hLine)

In [0]:
X_train_title = vectorizer_t.transform(X_train_title).toarray()

X_train_hLine = vectorizer_h.transform(X_train_hLine).toarray()

X_train_title.shape, X_train_hLine.shape



*   **Train_Validation Split : Splitting the training data into train data and validation data**

  *   Validation set plays a major role in improving the performance of the model.


In [0]:

X_train_t, X_valid_t, y_train_t, y_valid_t = train_test_split(X_train_title, Y_train_title, test_size=0.2, random_state=13)

X_train_h, X_valid_h, y_train_h, y_valid_h = train_test_split(X_train_hLine, Y_train_hLine, test_size=0.2, random_state=13)


# Model Building for Predictions



*   **TModel for predicting the SentimentTitle**
  *    Compile the TModel
  *    Summary of the TModel : Showing the architecture and Total Parameters of the TModel




In [0]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

In [0]:

Tmodel = Sequential()
Tmodel.add(Dense(128, activation='relu', input_dim=5000))
Tmodel.add(Dropout(0.2))
Tmodel.add(Dense(128, activation='relu'))
Tmodel.add(Dropout(0.2))
Tmodel.add(Dense(128, activation='relu'))
Tmodel.add(Dense(128, activation='relu'))
Tmodel.add(Dropout(0.2))
Tmodel.add(Dense(64, activation='relu'))
Tmodel.add(Dense(1,
                activation='linear',
                kernel_regularizer=L1L2(l1=0.02, l2=0.03)))
Tmodel.compile(loss='mean_absolute_error', optimizer= adam, metrics=['mean_absolute_error'])
Tmodel.summary()


*   **HModel for predicting the SentimentHeadLine**
  *    Compile the HModel
  *    Summary of the HModel : Showing the architecture and Total Parameters of the HModel


In [0]:

Hmodel = Sequential()
Hmodel.add(Dense(128, activation='relu', input_dim=5000))
Hmodel.add(Dropout(0.2))
Hmodel.add(Dense(128, activation='relu'))
Hmodel.add(Dropout(0.2))
Hmodel.add(Dense(128, activation='relu'))
Hmodel.add(Dense(128, activation='relu'))
Hmodel.add(Dense(64, activation='relu'))
Hmodel.add(Dense(1,
                activation='linear',
                kernel_regularizer=L1L2(l1=0.02, l2=0.03)))
Hmodel.compile(loss='mean_absolute_error', optimizer= adam, metrics=['mean_absolute_error'])
Hmodel.summary()

# Fitting the Train data to the Model



*   **Fitting the train data of title to the TModel for predicting the SentimentTitle** 



In [0]:
Tmodel.fit(X_train_t, y_train_t, batch_size = 1024, epochs = 30, validation_data=(X_valid_t, y_valid_t),verbose = True)


*   **Fitting the train data of title to the TModel for predicting the SentimentTitle**

In [0]:
Hmodel.fit(X_train_h, y_train_h, batch_size = 1024, epochs = 30, validation_data=(X_valid_h, y_valid_h),verbose = True)

# Conversion of Text Form Test Data into Vector Form

In [0]:
X_test_title = test_data['Title']
X_test_title = preprocess_data(X_test_title, lemma)
X_test_title = vectorizer_t.transform(X_test_title).toarray()


X_test_hLine = test_data['Headline']
X_test_hLine = preprocess_data(X_test_hLine, lemma)
X_test_hLine = vectorizer_h.transform(X_test_hLine).toarray()

# Predict the output of the Test Data

In [0]:
T_output = Tmodel.predict(X_test_title, 1024).flatten()

H_output = Hmodel.predict(X_test_hLine, 1024).flatten()

# Importing the results into the CSV File

In [0]:
test_data_df = pd.DataFrame({'IDLink': test_data['IDLink'],'SentimentTitle': T_output, 'SentimentHeadline':H_output})
test_data_df.to_csv('gdrive/My Drive/Approach_2_file.csv', index=False, columns=['IDLink','SentimentTitle', 'SentimentHeadline'])
test_data_df.to_csv('Output_file.csv', index=False, columns=['IDLink','SentimentTitle', 'SentimentHeadline'])